In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

GPU is available. Using GPU: Tesla T4


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split


path = "/content/drive/MyDrive/Data_Image"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Image_dataset = datasets.ImageFolder(root=path, transform=transform)


In [ ]:
# Chia dữ liệu thành tập train và tập test
train_dataset, test_dataset = random_split(Image_dataset, [0.7, 0.3])

batch_size = 32
# Tạo DataLoader để nạp dữ liệu theo batch
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
len(Image_dataset)

1346

In [ ]:
from torchvision.models import mobilenet_v2
import torch.nn as nn

class VisionModel(nn.Module):
    def __init__(self, num_classes=16):
        super(VisionModel, self).__init__()

        self.mobilenet_v2 = mobilenet_v2(pretrained=False).to(device)

        n_inputs = self.mobilenet_v2.classifier[1].in_features
        self.mobilenet_v2.classifier[1] = nn.Sequential(
            nn.Linear(n_inputs, num_classes, bias=True),
            nn.LogSoftmax(dim=1)
        )

    def forward(self, x):
        x = self.mobilenet_v2(x)
        return x

num_classes = len(Image_dataset.classes)
vision_model = VisionModel(num_classes).to(device)

In [ ]:
print(vision_model)


VisionModel(
  (mobilenet_v2): MobileNetV2(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 96, kernel_si

In [ ]:
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               accuracy_fn):
    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(data_loader):
        # Chuyển dữ liệu và mô hình lên GPU
        X, y = X.to(device), y.to(device)
        model.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        train_acc += accuracy_fn(y_true=y,
                                 y_pred=y_pred.argmax(dim=1)) # Go from logits -> pred labels

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

    # Calculate loss and accuracy per epoch and print out what's happening
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    print(f"Train loss: {train_loss:.5f} | Train accuracy: {train_acc:.2f}%")

def test_step(data_loader: torch.utils.data.DataLoader,
              model: torch.nn.Module,
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0, 0
    model.eval() # put model in eval mode
    # Turn on inference context manager
    with torch.inference_mode():
        for (X, y) in data_loader:
            # Chuyển dữ liệu lên GPU
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred = model(X)

            # 2. Calculate loss and accuracy
            test_loss += loss_fn(test_pred, y).item()
            test_acc += accuracy_fn(y_true=y,
                y_pred=test_pred.argmax(dim=1) # Go from logits -> pred labels
            )

        # Adjust metrics and print out
        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
        print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(vision_model.parameters(), lr=0.0001)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()  # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [ ]:
torch.manual_seed(42)

epochs = 30
for epoch in range(epochs):
    print(f"Epoch: {epoch}\n---------")
    vision_model.train()

    train_step(data_loader=train_loader,
        model=vision_model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        accuracy_fn=accuracy_fn
    )

    vision_model.eval()
    test_step(data_loader=test_loader,
        model=vision_model,
        loss_fn=loss_fn,
        accuracy_fn=accuracy_fn
    )

Epoch: 0
---------
Train loss: 2.65689 | Train accuracy: 15.46%
Test loss: 2.86808 | Test accuracy: 12.11%

Epoch: 1
---------
Train loss: 2.50629 | Train accuracy: 20.71%
Test loss: 3.45136 | Test accuracy: 12.11%

Epoch: 2
---------
Train loss: 2.40174 | Train accuracy: 23.09%
Test loss: 2.64613 | Test accuracy: 15.56%

Epoch: 3
---------
Train loss: 2.24876 | Train accuracy: 27.88%
Test loss: 2.29927 | Test accuracy: 26.06%

Epoch: 4
---------
Train loss: 2.04808 | Train accuracy: 35.86%
Test loss: 2.07718 | Test accuracy: 35.36%

Epoch: 5
---------
Train loss: 1.84693 | Train accuracy: 41.76%
Test loss: 1.99881 | Test accuracy: 35.99%

Epoch: 6
---------
Train loss: 1.66986 | Train accuracy: 47.64%
Test loss: 1.92307 | Test accuracy: 40.42%

Epoch: 7
---------
Train loss: 1.51665 | Train accuracy: 52.06%
Test loss: 1.92146 | Test accuracy: 36.25%

Epoch: 8
---------
Train loss: 1.36947 | Train accuracy: 57.81%
Test loss: 1.79098 | Test accuracy: 44.34%

Epoch: 9
---------
Train los